In [2]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [98]:
train_data_location = '/dataset/korean_unsmile_dataset/unsmile_train_v1.0.tsv'
test_data_location = '/dataset/korean_unsmile_dataset/unsmile_valid_v1.0.tsv'
base_dir = os.getcwd()
train_df = pd.read_csv(base_dir + train_data_location, sep='\t')
test_df = pd.read_csv(base_dir + test_data_location, sep='\t')
train_df.head(5)

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0


In [84]:
def count(df):
    clean_data_num = df.sum()[-2]
    hatred_data_num = df.sum()[1:].sum() - clean_data_num
    print(f'hatred data: {hatred_data_num}, clean data: {clean_data_num}')

In [147]:
count(train_df)
count(test_df)

hatred data: 12678, clean data: 3739
hatred data: 3124, clean data: 935


In [22]:
indexes = []
for i in range(len(train_df)):
    data = train_df.iloc[i]
    if data[1:].sum() == 1:
        indexes.append(i)
train_one_hot_df = train_df.iloc[indexes]

count(train_one_hot_df)

hatred data: 9984, clean data: 3739


In [85]:
train_df.sum()[1:].sort_values()

개인지칭      315
기타 혐오     569
연령        603
지역       1052
성소수자     1141
종교       1181
남성       1347
여성/가족    1599
인종/국적    1728
악플/욕설    3143
clean    3739
dtype: object

-Sorted by recall
category		recall
개인지칭		0.13513513513513514
기타 혐오		0.23880597014925373
악플/욕설		0.4541984732824427
연령		0.5136986301369864
여성/가족		0.5710659898477157
남성		0.5808383233532934
인종/국적		0.5868544600938967
clean		0.5893048128342246
성소수자		0.6857142857142857
지역		0.7153846153846154
종교		0.8379310344827586

In [99]:
total = {}
categories = ['female', 'male', 'minor', 'race', 'age', 'region', 'religion', 'extra', 'curse', 'clean', 'individual']

for category, value in zip(categories, train_df.sum()[1:]):
    total[category] = value

train_df.columns = ['sentence'] + categories
train_df.head()

,sentence,female,male,minor,race,age,region,religion,extra,curse,clean,individual
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0


In [122]:
pruned_df = train_df.copy()

for category in categories:
    if category == 'clean':
        continue
    
    indexes = list()
    for i, _ in pruned_df.iterrows():
        data = pruned_df.loc[i]
        if data[category] == 1:
            indexes.append(i)

    mask = np.random.random(len(indexes)) > 0.5
    indexes = mask * indexes
    pruned_df.drop(indexes, inplace=True, errors='ignore')

pruned_df.head()

,sentence,female,male,minor,race,age,region,religion,extra,curse,clean,individual
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0
5,고향가서 피방가면 동네 부럴 친구들이랑은 뭐 거르는 거 없이 이야기하니까 막 말하게...,0,0,0,0,0,0,0,0,0,1,0
7,나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임,0,0,0,0,0,0,0,0,1,0,0


In [113]:
pruned_df.sum()[1:]

female         734
male           643
minor          531
race           723
age            233
region         461
religion       520
extra          215
curse         1576
clean         3738
individual      73
dtype: object

In [114]:
train_df.sum()[1:]

female        1599
male          1347
minor         1141
race          1728
age            603
region        1052
religion      1181
extra          569
curse         3143
clean         3739
individual     315
dtype: object

-Results of test data using a trained model (unbalanced dataset. 5:1)
category		precision		recall
여성/가족		0.6267409470752089		0.5710659898477157
남성		    0.776		            0.5808383233532934
성소수자		0.7868852459016393		0.6857142857142857
인종/국적		0.7225433526011561		0.5868544600938967
연령	     	0.6880733944954128		0.5136986301369864
지역		    0.7654320987654321		0.7153846153846154
종교	      	0.8073089700996677		0.8379310344827586
기타 혐오		0.48484848484848486		0.23880597014925373
악플/욕설		0.5352323838080959		0.4541984732824427
clean	     	0.6598802395209581		0.5893048128342246
개인지칭		0.3125	             	0.13513513513513514

average precision: 0.651404101556005 60 중반
average recall: 0.5371756118558735   50 초반

-Sorted by recall
category		recall
개인지칭		0.13513513513513514
기타 혐오		0.23880597014925373
악플/욕설		0.4541984732824427
연령		    0.5136986301369864
여성/가족		0.5710659898477157
남성		    0.5808383233532934
인종/국적		0.5868544600938967
clean		    0.5893048128342246
성소수자		0.6857142857142857
지역		    0.7153846153846154
종교		    0.8379310344827586

======================================================================

-Results of test data using a trained model (balanced dataset. 3:2)
category		precision		recall
여성/가족		0.6428571428571429		0.43315508021390375
남성		    0.6618705035971223		0.5974025974025974
성소수자		0.743801652892562		0.6382978723404256
인종/국적		0.811965811965812		0.5026455026455027
연령	    	0.782608695652174		0.4931506849315068
지역	    	0.7107438016528925		0.7226890756302521
종교	    	0.7870370370370371		0.7264957264957265
기타 혐오		0.28		            0.13725490196078433
악플/욕설		0.3851851851851852		0.27296587926509186
clean	    	0.721868365180467		0.7272727272727273
개인지칭		0.3333333333333333		0.05555555555555555

average precision: 0.6237519572139752
average recall: 0.48244414579218853

-Sorted by recall
category		recall
개인지칭		0.05555555555555555
기타 혐오		0.13725490196078433
악플/욕설		0.27296587926509186
여성/가족		0.43315508021390375
연령		    0.4931506849315068
인종/국적		0.5026455026455027
남성		    0.5974025974025974
성소수자		0.6382978723404256
지역		    0.7226890756302521
종교		    0.7264957264957265
clean		    0.7272727272727273

======================================================================

-Results of test data using a trained model (balanced dataset. 1:1)
category		precision		recall
여성/가족		0.6424050632911392		0.5152284263959391
남성		    0.664576802507837		0.6347305389221557
성소수자		0.8243243243243243		0.6535714285714286
인종/국적		0.7314814814814815		0.5563380281690141
연령		    0.6666666666666666		0.4246575342465753
지역		    0.7863247863247863		0.7076923076923077
종교	     	0.8432203389830508		0.6862068965517242
기타 혐오		0.46153846153846156		0.22388059701492538
악플/욕설		0.5222816399286988		0.3727735368956743
clean		    0.5972083748753739		0.6406417112299465
개인지칭		0.3684210526315789		0.0945945945945946

average precision: 0.6462226356866726
average recall: 0.5009377818440259

-Sorted by recall
category		recall
개인지칭		0.0945945945945946
기타 혐오		0.22388059701492538
악플/욕설		0.3727735368956743
연령		    0.4246575342465753
여성/가족		0.5152284263959391
인종/국적		0.5563380281690141
남성		    0.6347305389221557
clean	     	0.6406417112299465
성소수자		0.6535714285714286
종교		    0.6862068965517242
지역		    0.7076923076923077

=> 근데 오버피팅 왤캐 심하냐?

In [ ]:
# total test data metrics: precision and recall
batch_generator = generate_batches(test_data, 1, device=args.device)

y_preds = []
y_labels = []

for batch_index, (x, y) in enumerate(batch_generator):
    y_pred = model(x)
    y_pred = (torch.sigmoid(y_pred) > 0.5).int()
    y_preds.append(y_pred.detach().cpu().numpy().reshape(-1))
    y_labels.append(y.detach().cpu().numpy().reshape(-1))

y_preds = np.array(y_preds)
y_labels = np.array(y_labels)

precisions = []
recalls = []
col_num = len(vectorized_train_df.columns[1:])

for i in range(col_num):
    pred_col = y_preds[:, i]
    label_col = y_labels[:, i]
    
    # precision and recall
    if pred_col.sum() == 0:
        precision = -1 # undefined
    else:
        precision = (label_col[pred_col == 1] == 1).sum() / pred_col.sum()
    recall = (label_col[pred_col == 1] == 1).sum() / label_col.sum()
    
    precisions.append(precision)    
    recalls.append(recall)
    
print('-Results of test data using a trained model\ncategory\t\tprecision\t\trecall')
for category, precision, recall in zip(vectorized_train_df.columns[1:], precisions, recalls):
    if precision == -1:
        print(f'{category}\t\tundefined\t\t{recall}')
    else:
        print(f'{category}\t\t{precision}\t\t{recall}')


recall_dict = {}
for category, recall in zip(vectorized_train_df.columns[1:], recalls):
    recall_dict[category] = recall
print('\n-Sorted by recall\ncategory\t\trecall')
for category, recall in sorted(recall_dict.items(), key=lambda x: x[1]):
    print(f'{category}\t\t{recall}')

    
# total precision and recall
TP = y_labels[y_preds == 1].sum()
TP_plus_FP = y_preds.sum()
TP_plus_FN = 

# '개인지칭' 과 '기타 혐오' 의 성능이 매우 낮음,, 왜?
# 데이터 통계를 보면, '개인지칭', '기타 혐오', '연령' 순서대로 데이터 수가 가장 적다. -> 적은 데이터 수가 문제?
# 그런데 '연령' 보다 '악플/욕설' 데이터 수가 약 5배 많음에도 불구하고, recall 은 큰 차이 안남,,
# 반대로, recall 이 가장 높은 '지역' 이나 '종교' 데이터 수가 많은 것도 아님.